#### Read swot l2 pixel cloud data and write out to masked file.


In [1]:
import os
import numpy as np
import xarray as xr
import geopandas as gpd
from glob import glob
from utils.swot_data_mask import swot_pixc_mask, swot_raster_mask
from utils.functions import swot_raster_reproj


In [2]:
dir_pixc = 'data/siling_co-lake/swot-pixc'
dir_raster = 'data/siling_co-lake/raster'
path_lake_vec = 'data/siling_co-lake/hydrolake_siling_co.gpkg'


In [3]:
## read vector file of the lake.
lake_gdf = gpd.read_file(path_lake_vec)
lake_gdf


,Hylak_id,Lake_name,Country,Continent,Poly_src,Lake_type,Grand_id,Lake_area,Shore_len,Shore_dev,...,Vol_src,Depth_avg,Dis_avg,Res_time,Elevation,Slope_100,Wshd_area,Pour_long,Pour_lat,geometry
0,147,Siling,China,Asia,SWBD,1,0,1749.53,402.54,2.71,...,1,28.0,0.0,-1.0,4539,-1.0,29077.1,89.047917,31.777083,"MULTIPOLYGON (((88.97853 32.01195, 88.9806 32...."


In [4]:
lake_decrease_gdf = lake_gdf.copy()
lon_center = lake_decrease_gdf.bounds.mean(axis=1).values
utm_zone = np.floor(lon_center/6)+31
epsg_code = f'326{int(utm_zone[0])}'
lake_decrease_gdf = lake_decrease_gdf.to_crs(epsg=epsg_code)
lake_decrease_gdf['geometry'] = lake_decrease_gdf.geometry.buffer(-200)
lake_decrease_gdf = lake_decrease_gdf.to_crs(epsg=4326)


#### 1. SWOT pixel cloud: batch data read, mask and write out

In [5]:
## Check original .nc raster file.
paths_pixc = [p for p in glob(dir_pixc + '/*.nc') if '_masked' not in p and 'filter' not in p and 'height' not in p]
paths_pixc = sorted(paths_pixc)
print(len(paths_pixc))


9


In [ ]:
for i, path in enumerate(paths_pixc):
  print(f'input file: {i} {path}')
  # Define the output path
  path_masked = path.split('.')[0]+'_masked.nc'
  pixc_nc = xr.open_dataset(path, group='pixel_cloud')
  pixc_masked = swot_pixc_mask(pixc_nc=pixc_nc, 
                                vars_sel=['latitude', 'longitude', 'height', 
                                          'solid_earth_tide', 'pole_tide', 
                                          'load_tide_fes', 'iono_cor_gim_ka', 'geoid',
                                          ],
                                region_gdf=lake_decrease_gdf, 
                                path_masked=path_masked)


input file: 0 data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_015_217_210L_20240516T015222_20240516T015234_PIC0_01.nc
file written to data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_015_217_210L_20240516T015222_20240516T015234_PIC0_01_masked.nc
input file: 1 data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_016_217_210L_20240605T223729_20240605T223740_PIC0_01.nc


#### 2. SWOT raster data: batch data read, mask and write out

In [ ]:
## Check original .nc raster file.
paths_raster_file = [p for p in glob(dir_raster + '/*.nc') if 'Raster_' in p and 'masked' not in p]
paths_raster_file = sorted(paths_raster_file)
print(len(paths_raster_file))


9


In [ ]:
for i, path in enumerate(paths_raster_file):
    print(f'input file: {i} {path}')
    # Define the output path
    path_masked = path.split('.')[0]+'_masked.nc'
    raster_nc = xr.open_dataset(path, decode_times=False)
    raster_epsg = '326' + str(raster_nc.attrs['utm_zone_num'])
    raster_wgs84_nc = swot_raster_reproj(raster_nc, epsg_from=raster_epsg, epsg_to='4326')
    raster_vars = swot_raster_mask(raster_nc=raster_wgs84_nc, 
                                vars_sel=['x', 'y', 'wse','sig0', 'geoid', 'wse_qual'],
                                region_gdf=lake_decrease_gdf, 
                                path_masked=path_masked)


input file: 0 data/dianchi-lake/raster/SWOT_L2_HR_Raster_100m_UTM48R_N_x_x_x_009_049_099F_20240105T211638_20240105T211659_PIC0_01.nc
file read and written to data/dianchi-lake/raster/SWOT_L2_HR_Raster_100m_UTM48R_N_x_x_x_009_049_099F_20240105T211638_20240105T211659_PIC0_01_masked.nc
input file: 1 data/dianchi-lake/raster/SWOT_L2_HR_Raster_100m_UTM48R_N_x_x_x_010_049_099F_20240126T180144_20240126T180205_PIC0_01.nc
file read and written to data/dianchi-lake/raster/SWOT_L2_HR_Raster_100m_UTM48R_N_x_x_x_010_049_099F_20240126T180144_20240126T180205_PIC0_01_masked.nc
input file: 2 data/dianchi-lake/raster/SWOT_L2_HR_Raster_100m_UTM48R_N_x_x_x_012_049_099F_20240308T113153_20240308T113214_PIC0_01.nc
file read and written to data/dianchi-lake/raster/SWOT_L2_HR_Raster_100m_UTM48R_N_x_x_x_012_049_099F_20240308T113153_20240308T113214_PIC0_01_masked.nc
input file: 3 data/dianchi-lake/raster/SWOT_L2_HR_Raster_100m_UTM48R_N_x_x_x_014_049_099F_20240419T050203_20240419T050224_PIC0_01.nc
file read and w